# BedrockEmbeddings

[Amazon Bedrock](https://aws.amazon.com/bedrock/) 是一项完全托管的服务，通过单一API提供来自AI21 Labs、Anthropic、Cohere、Meta、Stability AI 和 Amazon等领先AI公司的高性能基础模型(FM)选择，并提供构建生成式AI应用所需的一整套功能，包括安全性、隐私保护和负责任的AI实践。

这将帮助您使用LangChain开始使用Amazon Bedrock的[嵌入模型](/docs/concepts/embedding_models)。有关`Bedrock`功能和配置选项的详细文档，请参阅[API参考](https://api.js.langchain.com/classes/langchain_aws.BedrockEmbeddings.html)。

## 概览
### 集成详情

| 类别 | 包 | 本地支持 | [Python支持](https://python.langchain.com/docs/integrations/text_embedding/bedrock/) | 包下载量 | 最新包版本 |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [Bedrock](https://api.js.langchain.com/classes/langchain_aws.BedrockEmbeddings.html) | [@langchain/aws](https://api.js.langchain.com/modules/langchain_aws.html) | ❌ | ✅ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/aws?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/aws?style=flat-square&label=%20&) |

## 准备工作

要访问Bedrock嵌入模型，您需要创建一个AWS账户，获取API密钥，并安装`@langchain/aws`集成包。

前往[AWS文档](https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html)注册AWS并设置您的凭证。您还需要为您的账户启用模型访问权限，可以通过[按照这些说明操作](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)来完成。

### 凭证

如果您希望自动追踪您的模型调用，也可以取消下面的注释来设置您的[LangSmith](https://docs.smith.langchain.com/) API密钥：

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### 安装

LangChain Bedrock集成位于`@langchain/aws`包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/aws @langchain/core
</Npm2Yarn>
```

## 实例化

现在我们可以实例化我们的模型对象并嵌入文本。

有几种不同的方式可以与AWS进行身份验证——以下示例依赖于在环境变量中设置的访问密钥、秘密访问密钥和区域：

In [1]:
import { BedrockEmbeddings } from "@langchain/aws";

const embeddings = new BedrockEmbeddings({
  region: process.env.BEDROCK_AWS_REGION!,
  credentials: {
    accessKeyId: process.env.BEDROCK_AWS_ACCESS_KEY_ID!,
    secretAccessKey: process.env.BEDROCK_AWS_SECRET_ACCESS_KEY!,
  },
  model: "amazon.titan-embed-text-v1",
});

## 索引与检索

嵌入模型通常用于检索增强生成（RAG）流程中，既用于数据索引，也用于后续的数据检索。有关更详细的说明，请参阅我们位于[使用外部知识教程](/docs/tutorials/#working-with-external-knowledge)下的RAG教程。

下面展示如何使用我们之前初始化的`embeddings`对象进行数据索引和检索。在此示例中，我们将使用演示版的[`MemoryVectorStore`](/docs/integrations/vectorstores/memory)来索引和检索一个示例文档。

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## 直接使用

在底层，向量存储和检索器的实现分别通过调用 `embeddings.embedDocument(...)` 和 `embeddings.embedQuery(...)` 来为 `fromDocuments` 和检索器的 `invoke` 操作中使用的文本创建嵌入。

您可以直接调用这些方法，以用于您自己的用例。

### 嵌入单个文本

您可以使用 `embedQuery` 对查询进行嵌入，以用于搜索。这将生成针对该查询的向量表示：

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
         0.625,  0.111328125,      0.265625,   -0.20019531,  0.40820312,
  -0.010803223,  -0.22460938, -0.0002937317,    0.29882812, -0.14355469,
  -0.068847656,   -0.3984375,          0.75,    -0.1953125,  -0.5546875,
  -0.087402344,       0.5625,      1.390625,    -0.3515625,  0.39257812,
  -0.061767578,      0.65625,   -0.36328125,   -0.06591797,    0.234375,
   -0.36132812,   0.42382812,  -0.115234375,   -0.28710938, -0.29296875,
     -0.765625,  -0.16894531,    0.23046875,     0.6328125, -0.08544922,
    0.13671875, 0.0004272461,        0.3125,    0.12207031,   -0.546875,
    0.14257812, -0.119628906,  -0.111328125,    0.61328125,      0.6875,
     0.3671875,   -0.2578125,   -0.27734375,      0.703125,    0.203125,
    0.17675781,  -0.26757812,   -0.76171875,    0.71484375,  0.77734375,
    -0.1953125, -0.007232666,  -0.044921875,    0.23632812, -0.24121094,
  -0.012207031,    0.5078125,    0.08984375,    0.56640625,  -0.3046875,
     0.6484375,        -0.25,   -0.37890625,    -

### 嵌入多个文本

您可以使用 `embedDocuments` 对多个文本进行索引嵌入。此方法的内部实现可能（但不一定）与嵌入查询的不同：

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
         0.625,  0.111328125,      0.265625,   -0.20019531,  0.40820312,
  -0.010803223,  -0.22460938, -0.0002937317,    0.29882812, -0.14355469,
  -0.068847656,   -0.3984375,          0.75,    -0.1953125,  -0.5546875,
  -0.087402344,       0.5625,      1.390625,    -0.3515625,  0.39257812,
  -0.061767578,      0.65625,   -0.36328125,   -0.06591797,    0.234375,
   -0.36132812,   0.42382812,  -0.115234375,   -0.28710938, -0.29296875,
     -0.765625,  -0.16894531,    0.23046875,     0.6328125, -0.08544922,
    0.13671875, 0.0004272461,        0.3125,    0.12207031,   -0.546875,
    0.14257812, -0.119628906,  -0.111328125,    0.61328125,      0.6875,
     0.3671875,   -0.2578125,   -0.27734375,      0.703125,    0.203125,
    0.17675781,  -0.26757812,   -0.76171875,    0.71484375,  0.77734375,
    -0.1953125, -0.007232666,  -0.044921875,    0.23632812, -0.24121094,
  -0.012207031,    0.5078125,    0.08984375,    0.56640625,  -0.3046875,
     0.6484375,        -0.25,   -0.37890625,    -

## 配置 Bedrock 运行时客户端

如果您想要自定义诸如
`credentials`（凭证）、`region`（区域）、`retryPolicy`（重试策略）等选项，可以传入自己实例化的 `BedrockRuntimeClient`。

In [ ]:
import { BedrockRuntimeClient } from "@aws-sdk/client-bedrock-runtime";
import { BedrockEmbeddings } from "@langchain/aws";

const getCredentials = () => {
  // do something to get credentials
}

// @lc-ts-ignore
const client = new BedrockRuntimeClient({
  region: "us-east-1",
  credentials: getCredentials(),
});

const embeddingsWithCustomClient = new BedrockEmbeddings({
  client,
});

## API 参考文档

有关所有 Bedrock 功能和配置的详细文档，请访问 API 参考页面: https://api.js.langchain.com/classes/langchain_aws.BedrockEmbeddings.html